In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import matplotlib.patches as mpatches

from matplotlib.collections import PatchCollection
import matplotlib

In [ ]:
#list of number of link that are assigned
linkSummary = pd.read_csv('geomInfo/ModuleLinkSummary.csv')
linkSummary = linkSummary[(linkSummary.Cassette==0) & (linkSummary.Layer>=5) & (linkSummary.Layer<=9)]
linkSummary.set_index(['Layer','ModU','ModV'],inplace=True)


In [ ]:
#load data from csv files.  This is used to extract average data sizes

daq_Data = pd.concat([pd.read_csv(f'Data/ttbar_DAQ_data_{i}.csv') for i in range(16)])


In [ ]:
#get list of wedges, and assign x and y positions for drawing maps
wedges = daq_Data.groupby(['layer','waferu','waferv']).any()[['HDM']].reset_index()

wedges['y'] = wedges.waferv
wedges['x'] = 0
wedges.loc[:,'x'] = (2+wedges.y-2*wedges.waferu)
wedges.y *= 1.5
wedges.x *= -3**.5/2
wedges.set_index(['layer','waferu','waferv'],inplace=True)

In [ ]:
#function to look through output logs of condor jobs to extract and accumulate the results across several runs
# if you just ran interactivel, need to just get the info on the overflows, max size, etc into a dataframe.

def getDF(inputDF, LogName=None,Nstart=0,N=100):
    fullData = inputDF.groupby(['layer','waferu','waferv']).mean()
    fullData['eTx_assigned'] = linkSummary.ECOND_eTX
    fullData.eTx_assigned = fullData.eTx_assigned.fillna(1).astype(int)

    fullData['eTx_30percent'] = np.ceil((fullData.TotalWords/53.3333)*1.3).astype(int)
    fullData['eTx_Mean'] = np.ceil((fullData.TotalWords/53.3333))
    
    if not LogName is None:
        data_overflow = []
        data_maxSize = []
        L1As_issued=0
        fileName = f'LogFiles/{LogName}_{Nstart}.stdout'
        with open(fileName,'r') as _file:
            for line in _file:
                if 'overflows= ' in line:
                    data_overflow.append(eval(line[10:]))
                if 'maxSize= ' in line:
                    data_maxSize.append(eval(line[8:]))
                if 'L1As issued' in line:
                    L1As_issued += int(line.split()[0])
                    
        data_overflow=np.array(data_overflow)
        data_maxSize=np.array(data_maxSize)

        for i_file in range(Nstart+1,Nstart+N):
            fileName = f'LogFiles/{LogName}_{i_file}.stdout'
            with open(fileName,'r') as _file:
                for line in _file:
                    if 'eTx' in line:
                        i = int(line.split()[0])-1
                    if 'overflows= ' in line:
                        overflow=np.array(eval(line[10:]))
                        overflow[overflow<0] = 99999999
                        data_overflow[i] += overflow
                    if 'maxSize= ' in line:
                        maxSize=np.array(eval(line[8:]))
                        data_maxSize[i] = np.maximum(data_maxSize[i], maxSize)
                    if 'L1As issued' in line:
                        L1As_issued += int(line.split()[0])
                        
        x = ((data_overflow.transpose()==0)*np.arange(1,7))
        x[x==0]=99
        minLinks = x.min(axis=1)
        fullData['min_eTx'] = minLinks

        nLinks = minLinks
        maxSize = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
        fullData['maxSize'] = maxSize    

        
        nLinks = fullData.eTx_assigned.values
        maxSize = []
        overflows = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
            overflows.append(data_overflow.transpose()[i][nLinks[i]-1])
        fullData['maxSize_assigned'] = maxSize
        fullData['overflows_assigned'] = overflows


        nLinks = fullData.eTx_30percent.values
        maxSize = []
        overflows = []
        for i in range(len(nLinks)):
            maxSize.append(data_maxSize.transpose()[i][nLinks[i]-1])
            overflows.append(data_overflow.transpose()[i][nLinks[i]-1])
        fullData['maxSize_30percent'] = maxSize
        fullData['overflows_30percent'] = overflows
    return fullData, L1As_issued

In [ ]:
fullData,L1As = getDF(daq_Data,LogName='DAQBufferStudy_69415084',Nstart=0,N=100)
print(L1As)


In [ ]:
fullData.head()

In [ ]:
#dictionaly of the labels for different types of plots
Labels ={'occ':{'Title':'Average L1A Occupancy, Layer %i',
                'colorLabel':'Avg. Occ. Above Zero Suppression',
                'nDec':1,
                'zMax':260},
         'TotalWords':{'Title':'Average L1A Size (32b Words), Layer %i',
                      'colorLabel':'Avg. L1A Size (32b words)',
                      'nDec':1,
                      'zMax':200},
         'EmptyLinks':{'Title':'Average Number of Empty eRx Packets, Layer %i',
                      'colorLabel':'Avg. # of Empty eRx Packets',
                      'nDec':1,
                      'zMax':6},
         'eTx_assigned':{'Title':'Number of eTx assigned, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'eTx_30percent':{'Title':'Number of eTx assuming 30 percent overhead, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'eTx_Mean':{'Title':'Number of eTx based on mean data, no rounding, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':2,
                      'zMax':6},
         'min_eTx':{'Title':'Min number of eTx with 0 overflows, Layer %i',
                      'colorLabel':'# of eTx',
                      'nDec':0,
                      'zMax':6},
         'maxSize':{'Title':'Max buffer size, with 0 overflows, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'maxSize_assigned':{'Title':'Max buffer size, using assigned number of links, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'overflows_assigned':{'Title':'Overflow counter, using assigned number of links, Layer %i',
                      'colorLabel':'# of buffer overflows',
                      'nDec':0,
                      'zMax':40000},
         'maxSize_30percent':{'Title':'Max buffer size, using link assignment from 30 percent overhead, Layer %i',
                      'colorLabel':'Buffer size (32b words)',
                      'nDec':0,
                      'zMax':1600},
         'overflows_30percent':{'Title':'Overflow counter, using link assignment from 30 percent overhead, Layer %i',
                      'colorLabel':'# of buffer overflows',
                      'nDec':0,
                      'zMax':40000},
        }

In [ ]:
def makePlot(var,patches,df=fullData,extra=""):
    data = df.loc[layer,var].values.flatten()

    waferCollection = PatchCollection(patches,cmap=matplotlib.cm.coolwarm)
    waferCollection.set_array(data)
    waferCollection.set_clim([0,Labels[var]['zMax']])
    fig, ax = plt.subplots(figsize=(15,10))
    ax.add_collection(waferCollection)
    plt.axis([0,16,-1,15])
    # ax.set_xticklabels([])
    # ax.set_yticklabels([]);
    cbar = plt.colorbar(waferCollection)
    cbar.set_label(Labels[var]['colorLabel'],fontsize=14)

    wedge['data'] = data

    plt.title(Labels[var]['Title']%layer,fontsize=24)
    for x,y,d in wedge[['x','y','data']].values:
    #     s = data[i]
        plt.text(x,y,f'%.{Labels[var]["nDec"]}f'%d,fontsize=14,horizontalalignment='center',verticalalignment='center')

                 

    fig.savefig(f'Plots/{var}_layer{layer}{extra}.pdf')

In [ ]:

for layer in [5,7,9]:
    wedge = wedges.loc[layer]
    patches = []

    for w in wedge.index:
        patches.append(mpatches.RegularPolygon((wedge.loc[w].x,wedge.loc[w].y),6,.95))
    for k in Labels:
        print(k)
        makePlot(k,df=fullData,extra="_OldTTbarTests",patches=patches)
